Pobranie wszystkich bibliotek potrzebnych do analizy

In [1004]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import dalex as dx

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import warnings
warnings.filterwarnings('ignore')

Zaimportowanie danych wraz z krótką analizą

In [1005]:
df = pd.read_csv('aug_train.csv', sep=',', decimal='.')
sampled_df = df.drop(['enrollee_id', 'city'], axis=1).sample(frac=0.05)
display(sampled_df)

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
1586,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,2,50-99,Pvt Ltd,1,43,0.0
6725,0.920,NaN,Has relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,1,79,0.0
11514,0.926,NaN,No relevent experience,no_enrollment,Phd,STEM,>20,10000+,Pvt Ltd,>4,18,1.0
6312,0.843,NaN,Has relevent experience,no_enrollment,Graduate,STEM,9,100-500,NGO,1,130,0.0
11364,0.804,NaN,Has relevent experience,Part time course,Graduate,STEM,8,10/49,Pvt Ltd,1,17,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3646,0.550,Male,Has relevent experience,Part time course,Graduate,STEM,2,10/49,Early Stage Startup,1,41,0.0
10342,0.920,Male,No relevent experience,Full time course,Graduate,STEM,14,NaN,NaN,never,88,0.0
664,0.804,NaN,Has relevent experience,NaN,Graduate,STEM,12,5000-9999,Pvt Ltd,1,182,0.0
10580,0.884,Male,Has relevent experience,no_enrollment,Masters,STEM,9,50-99,Public Sector,3,166,0.0


In [1006]:
sampled_df.head()

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
1586,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,2,50-99,Pvt Ltd,1,43,0.0
6725,0.920,NaN,Has relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,1,79,0.0
11514,0.926,NaN,No relevent experience,no_enrollment,Phd,STEM,>20,10000+,Pvt Ltd,>4,18,1.0
6312,0.843,NaN,Has relevent experience,no_enrollment,Graduate,STEM,9,100-500,NGO,1,130,0.0
11364,0.804,NaN,Has relevent experience,Part time course,Graduate,STEM,8,10/49,Pvt Ltd,1,17,0.0


In [1007]:
sampled_df.dtypes

city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                    float64
dtype: object

In [1008]:
sampled_df.describe()

,city_development_index,training_hours,target
count,958.000000,958.000000,958.000000
mean,0.825101,68.824635,0.278706
std,0.125529,61.698821,0.448597
min,0.479000,2.000000,0.000000
25%,0.738000,24.000000,0.000000
50%,0.897000,52.000000,0.000000
75%,0.920000,94.000000,1.000000
max,0.949000,332.000000,1.000000


Sprawdzenie ilości brakujących danych

In [1009]:
sampled_df.isna().sum()

city_development_index      0
gender                    258
relevent_experience         0
enrolled_university        19
education_level            25
major_discipline          169
experience                  4
company_size              315
company_type              312
last_new_job               20
training_hours              0
target                      0
dtype: int64

Zastąpienie braków danych nową kolumną 'No data'

In [1010]:
sampled_df['gender'].fillna('No data', inplace=True)
sampled_df['enrolled_university'].fillna('No data', inplace=True)
sampled_df['education_level'].fillna('No data', inplace=True)
sampled_df['major_discipline'].fillna('No data', inplace=True)
sampled_df['experience'].fillna('No data', inplace=True)
sampled_df['company_size'].fillna('No data', inplace=True)
sampled_df['company_type'].fillna('No data', inplace=True)
sampled_df['last_new_job'].fillna('No data', inplace=True)

Wyświetlanie zmiennych unikalnych dla każdej wartości

In [1011]:
sampled_df.describe(include=['object'])

,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job
count,958,958,958,958,958,958,958,958,958
unique,4,2,4,6,7,23,9,7,7
top,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,No data,Pvt Ltd,1
freq,647,672,670,571,699,151,315,483,424


In [1012]:
sampled_df['gender'].value_counts()

Male       647
No data    258
Female      45
Other        8
Name: gender, dtype: int64

In [1013]:
sampled_df['enrolled_university'].value_counts()

no_enrollment       670
Full time course    198
Part time course     71
No data              19
Name: enrolled_university, dtype: int64

In [1014]:
sampled_df['education_level'].value_counts()

Graduate          571
Masters           205
High School       128
No data            25
Primary School     15
Phd                14
Name: education_level, dtype: int64

In [1015]:
sampled_df['major_discipline'].value_counts()

STEM               699
No data            169
Humanities          28
Other               22
Business Degree     20
No Major            10
Arts                10
Name: major_discipline, dtype: int64

In [1016]:
sampled_df['experience'].value_counts()

>20        151
5           80
4           77
3           71
2           65
10          60
7           56
9           50
6           46
1           35
8           34
14          29
11          28
15          26
16          26
12          23
<1          22
17          20
13          20
19          14
20          12
18           9
No data      4
Name: experience, dtype: int64

In [1017]:
sampled_df['company_size'].value_counts()

No data      315
50-99        150
100-500      131
10000+        98
1000-4999     74
<10           73
10/49         63
500-999       31
5000-9999     23
Name: company_size, dtype: int64

In [1018]:
sampled_df['company_type'].value_counts()

Pvt Ltd                483
No data                312
Public Sector           52
Early Stage Startup     37
Funded Startup          36
NGO                     28
Other                   10
Name: company_type, dtype: int64

In [1019]:
sampled_df['last_new_job'].value_counts()

1          424
>4         145
2          145
never      141
4           42
3           41
No data     20
Name: last_new_job, dtype: int64

In [1020]:
sampled_df.isna().sum()

city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
target                    0
dtype: int64

In [1021]:
sampled_df['target'].value_counts()

0.0    691
1.0    267
Name: target, dtype: int64

Przydzielenie zmiennych numerycznych i kategorycznych do odzielnych grup wraz z enkodowaniem zmiennych kategorycznych oraz standaryzacją zmiennych numerycznych

In [1022]:
categorical_columns = ['gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job']
numerical_columns = [col for col in sampled_df.columns if col not in categorical_columns + ['target']]
x_numeric = sampled_df[numerical_columns]
x_categorical = sampled_df[categorical_columns]

encoder = LabelEncoder()
x_categorical_encoded = x_categorical.apply(encoder.fit_transform)

scaler = StandardScaler()
x_numeric_standardized = pd.DataFrame(scaler.fit_transform(x_numeric), columns=x_numeric.columns)

X = pd.concat([x_numeric, x_categorical_encoded], axis=1)
sampled_df.drop(['target'], axis=1)
y = sampled_df['target']

Wyznaczenie zmiennych do zbioru uczącego

In [1023]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [1024]:
udzial_pozytywny_train = y_train.sum() / len(y_train)
udzial_negatywny_train = 1 - udzial_pozytywny_train
print(f'Udział negatywny: {udzial_negatywny_train:.4f}')
print(f'Udział pozytywny: {udzial_pozytywny_train:.4f}')

Udział negatywny: 0.7206
Udział pozytywny: 0.2794


Wykorzystanie modelu regresji logistycznej do predykcji wyników z wykorzystaniem oversamplingu

In [1025]:
logistic_regression_over = LogisticRegression(random_state=42, max_iter=1000)
oversample = RandomOverSampler(sampling_strategy={1: (50*len(y_train)), 0: (50*len(y_train))})
X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)

logistic_regression_over.fit(X_train_over, y_train_over)
y_pred_over = logistic_regression_over.predict(X_test)
accuracy_over = accuracy_score(y_test, y_pred_over)
print(f'Dokładność po oversamplingu {accuracy_over:.2%}')
print(f'Raport klasyfikacyjny po oversamplingu: {classification_report(y_test, y_pred_over)}')

conf_matrix = confusion_matrix(y_test, y_pred_over)
true_negatives = conf_matrix[0, 0]
false_positives = conf_matrix[0, 1]

specificity = true_negatives / (true_negatives + false_positives)
print(f'Specificity: {specificity}')

true_positives = conf_matrix[1, 1]
false_negatives = conf_matrix[1, 0]

sensitivity = true_positives / (true_positives + false_negatives)
print(f'Sensitivity: {sensitivity}')

Dokładność po oversamplingu 75.00%
Raport klasyfikacyjny po oversamplingu:               precision    recall  f1-score   support

         0.0       0.90      0.74      0.81       139
         1.0       0.53      0.77      0.63        53

    accuracy                           0.75       192
   macro avg       0.71      0.76      0.72       192
weighted avg       0.80      0.75      0.76       192

Specificity: 0.7410071942446043
Sensitivity: 0.7735849056603774


Wykorzystanie modelu drzew decyzyjnych do predykcji wyników z wykorzystaniem oversamplingu

In [1026]:
random_forest_over = DecisionTreeClassifier(max_depth=1)
oversample = RandomOverSampler(sampling_strategy = {1: (50*len(y_train)), 0: (50*len(y_train))})
X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)

random_forest_over.fit(X_train_over, y_train_over)
y_pred_over = random_forest_over.predict(X_test)
accuracy_over = accuracy_score(y_test, y_pred_over)
print(f'Dokładność po oversamplingu {accuracy_over:.2%}')
print(f'Raport klasyfikacyjny po oversamplingu: {classification_report(y_test, y_pred_over)}')

conf_matrix = confusion_matrix(y_test, y_pred_over)
true_negatives = conf_matrix[0, 0]
false_positives = conf_matrix[0, 1]

specificity = true_negatives / (true_negatives + false_positives)
print(f'Specificity: {specificity}')

true_positives = conf_matrix[1, 1]
false_negatives = conf_matrix[1, 0]

sensitivity = true_positives / (true_positives + false_negatives)
print(f'Sensitivity: {sensitivity}')

Dokładność po oversamplingu 77.60%
Raport klasyfikacyjny po oversamplingu:               precision    recall  f1-score   support

         0.0       0.82      0.89      0.85       139
         1.0       0.62      0.47      0.54        53

    accuracy                           0.78       192
   macro avg       0.72      0.68      0.69       192
weighted avg       0.76      0.78      0.77       192

Specificity: 0.8920863309352518
Sensitivity: 0.4716981132075472


Wykorzystanie modelu lasów losowych do predykcji wyników z wykorzystaniem oversamplingu

In [1027]:
random_forest_over = RandomForestClassifier(n_estimators=350, max_features=5, random_state=42)
oversample = RandomOverSampler(sampling_strategy = {1: (50*len(y_train)), 0: (50*len(y_train))})
X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)

random_forest_over.fit(X_train_over, y_train_over)
y_pred_over = random_forest_over.predict(X_test)
accuracy_over = accuracy_score(y_test, y_pred_over)
print(f'Dokładność po oversamplingu {accuracy_over:.2%}')
print(f'Raport klasyfikacyjny po oversamplingu: {classification_report(y_test, y_pred_over)}')

conf_matrix = confusion_matrix(y_test, y_pred_over)
true_negatives = conf_matrix[0, 0]
false_positives = conf_matrix[0, 1]

specificity = true_negatives / (true_negatives + false_positives)
print(f'Specificity: {specificity}')

true_positives = conf_matrix[1, 1]
false_negatives = conf_matrix[1, 0]

sensitivity = true_positives / (true_positives + false_negatives)
print(f'Sensitivity: {sensitivity}')

Dokładność po oversamplingu 75.52%
Raport klasyfikacyjny po oversamplingu:               precision    recall  f1-score   support

         0.0       0.80      0.88      0.84       139
         1.0       0.57      0.43      0.49        53

    accuracy                           0.76       192
   macro avg       0.69      0.66      0.67       192
weighted avg       0.74      0.76      0.74       192

Specificity: 0.8776978417266187
Sensitivity: 0.4339622641509434


Interpretowalność

In [1028]:
categorical_features = sampled_df.select_dtypes(exclude=np.number).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, ["city_development_index", "training_hours"]),
        ('cat', categorical_transformer, categorical_features)])

In [1029]:
X = sampled_df.drop(['target'], axis=1)
y = sampled_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', LogisticRegression(max_iter=1000))])  # Zwiększenie liczby iteracji, jeśli to konieczne
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['city_development_index',
                                                   'training_hours']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['gender',
                                                   'relevent_experience',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'experience', 'company_size',
                                                   'company_type',
                                                   'last_new_job'])])),
                ('classifier', LogisticRegression(max_iter=1000))])

Profile ceteris-paribus (PCP)

In [1030]:
exp = dx.Explainer(model, X_train, y_train, label="Logistic Regression")

obs = X_train.iloc[2]

pcp = exp.predict_profile(obs)

pcp.plot(variables=["city_development_index", "training_hours"])

pcp.plot(variables=["gender", "relevent_experience", "enrolled_university", "education_level", "major_discipline", "experience", "comapny_size", "company_type", "last_new_job"], variable_type = "categorical")

Preparation of a new explainer is initiated

  -> data              : 766 rows 11 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 766 values
  -> model_class       : sklearn.linear_model._logistic.LogisticRegression (default)
  -> label             : Logistic Regression
  -> predict function  : <function yhat_proba_default at 0x000001C7654D28C0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.00554, mean = 0.279, max = 0.949
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.919, mean = -6.41e-07, max = 0.953
  -> model_info        : package sklearn

A new explainer has been created!


Calculating ceteris paribus:   0%|          | 0/11 [00:00<?, ?it/s]

Calculating ceteris paribus: 100%|██████████| 11/11 [00:00<00:00, 129.42it/s]


In [1031]:
gender_encoder = LabelEncoder().fit(sampled_df['gender'])
print(dict(zip(gender_encoder.classes_, gender_encoder.transform(gender_encoder.classes_))))

{'Female': 0, 'Male': 1, 'No data': 2, 'Other': 3}


In [1032]:
relevent_experience_encoder = LabelEncoder().fit(sampled_df['relevent_experience'])
print(dict(zip(relevent_experience_encoder.classes_, relevent_experience_encoder.transform(relevent_experience_encoder.classes_))))

{'Has relevent experience': 0, 'No relevent experience': 1}


In [1033]:
university_encoder = LabelEncoder().fit(df['enrolled_university'])
print(dict(zip(university_encoder.classes_, university_encoder.transform(university_encoder.classes_))))

{'Full time course': 0, 'Part time course': 1, 'no_enrollment': 2, nan: 3}


In [1034]:
education_encoder = LabelEncoder().fit(df['education_level'])
print(dict(zip(education_encoder.classes_, education_encoder.transform(education_encoder.classes_))))

{'Graduate': 0, 'High School': 1, 'Masters': 2, 'Phd': 3, 'Primary School': 4, nan: 5}


In [1035]:
discipline_encoder = LabelEncoder().fit(df['major_discipline'])
print(dict(zip(discipline_encoder.classes_, discipline_encoder.transform(discipline_encoder.classes_))))

{'Arts': 0, 'Business Degree': 1, 'Humanities': 2, 'No Major': 3, 'Other': 4, 'STEM': 5, nan: 6}


In [1036]:
experience_encoder = LabelEncoder().fit(df['experience'])
print(dict(zip(experience_encoder.classes_, experience_encoder.transform(experience_encoder.classes_))))

{'1': 0, '10': 1, '11': 2, '12': 3, '13': 4, '14': 5, '15': 6, '16': 7, '17': 8, '18': 9, '19': 10, '2': 11, '20': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, '<1': 20, '>20': 21, nan: 22}


In [1037]:
company_size_encoder = LabelEncoder().fit(df['company_size'])
print(dict(zip(company_size_encoder.classes_, company_size_encoder.transform(company_size_encoder.classes_))))

{'10/49': 0, '100-500': 1, '1000-4999': 2, '10000+': 3, '50-99': 4, '500-999': 5, '5000-9999': 6, '<10': 7, nan: 8}


In [1038]:
comapny_type_encoder = LabelEncoder().fit(df['company_type'])
print(dict(zip(comapny_type_encoder.classes_, comapny_type_encoder.transform(comapny_type_encoder.classes_))))

{'Early Stage Startup': 0, 'Funded Startup': 1, 'NGO': 2, 'Other': 3, 'Public Sector': 4, 'Pvt Ltd': 5, nan: 6}


In [1039]:
job_encoder = LabelEncoder().fit(df['last_new_job'])
print(dict(zip(job_encoder.classes_, job_encoder.transform(job_encoder.classes_))))

{'1': 0, '2': 1, '3': 2, '4': 3, '>4': 4, 'never': 5, nan: 6}


Wykresy częściowej zależności (PDP)

In [1040]:
pdp_training_hours = exp.model_profile(variables="training_hours")
pdp_training_hours.plot()

Calculating ceteris paribus: 100%|██████████| 1/1 [00:00<00:00, 12.25it/s]


In [1041]:
pdp_training_hours = exp.model_profile(variables="city_development_index")
pdp_training_hours.plot()

Calculating ceteris paribus: 100%|██████████| 1/1 [00:00<00:00, 13.01it/s]


In [1042]:
X_train['gender_text'] = X_train['training_hours'].map({'Female': 0, 'Male': 1, 'No data': 2, 'Other': 3})

# Użycie tej kolumny do grupowania
pdp_age_gender = exp.model_profile(variables=['training_hours'], groups='gender')
pdp_age_gender.plot(title="PDP for training_hours with gender groups")

Calculating ceteris paribus: 100%|██████████| 1/1 [00:00<00:00,  8.93it/s]


In [1043]:
X_train['gender_text'] = X_train['city_development_index'].map({'Female': 0, 'Male': 1, 'No data': 2, 'Other': 3})

# Użycie tej kolumny do grupowania
pdp_age_gender = exp.model_profile(variables=['city_development_index'], groups='gender')
pdp_age_gender.plot(title="PDP for training_hours with gender groups")

Calculating ceteris paribus: 100%|██████████| 1/1 [00:00<00:00, 12.99it/s]


In [1044]:
X_train['education_level_text'] = X_train['training_hours'].map({'Primary School': 0, 'High School': 1, 'Graduate': 2, 'Masters': 3, 'Phd': 4, 'No data': 5})

# Użycie tej kolumny do grupowania
pdp_age_gender = exp.model_profile(variables=['training_hours'], groups='education_level')
pdp_age_gender.plot(title="PDP for training_hours with gender groups")

Calculating ceteris paribus: 100%|██████████| 1/1 [00:00<00:00, 14.71it/s]


In [1045]:
X_train['education_level_text'] = X_train['city_development_index'].map({'Primary School': 0, 'High School': 1, 'Graduate': 2, 'Masters': 3, 'Phd': 4, 'No data': 5})

# Użycie tej kolumny do grupowania
pdp_age_gender = exp.model_profile(variables=['city_development_index'], groups='education_level')
pdp_age_gender.plot(title="PDP for training_hours with gender groups")

Calculating ceteris paribus: 100%|██████████| 1/1 [00:00<00:00, 10.53it/s]


In [1046]:
pdp_gender = exp.model_profile(variables=['gender'], variable_type='categorical')
pdp_gender.plot(title="PDP for gender")

Calculating ceteris paribus: 100%|██████████| 1/1 [00:00<00:00, 83.34it/s]


In [1047]:
pdp_experience = exp.model_profile(variables=['relevent_experience'], variable_type='categorical')
pdp_experience.plot(title="PDP for relevent_experience")

Calculating ceteris paribus: 100%|██████████| 1/1 [00:00<00:00, 99.98it/s]


In [1048]:
pdp_experience = exp.model_profile(variables=['enrolled_university'], variable_type='categorical')
pdp_experience.plot(title="PDP for enrolled_university")

Calculating ceteris paribus: 100%|██████████| 1/1 [00:00<00:00, 76.93it/s]


In [1049]:
pdp_experience = exp.model_profile(variables=['education_level'], variable_type='categorical')
pdp_experience.plot(title="PDP for education_level")

Calculating ceteris paribus: 100%|██████████| 1/1 [00:00<00:00, 83.30it/s]


In [1050]:
pdp_experience = exp.model_profile(variables=['major_discipline'], variable_type='categorical')
pdp_experience.plot(title="PDP for major_discipline")

Calculating ceteris paribus: 100%|██████████| 1/1 [00:00<00:00, 83.33it/s]


In [1051]:
pdp_experience = exp.model_profile(variables=['experience'], variable_type='categorical')
pdp_experience.plot(title="PDP for experience")

Calculating ceteris paribus: 100%|██████████| 1/1 [00:00<00:00, 38.46it/s]


In [1052]:
pdp_experience = exp.model_profile(variables=['company_size'], variable_type='categorical')
pdp_experience.plot(title="PDP for comapny_size")

Calculating ceteris paribus: 100%|██████████| 1/1 [00:00<00:00, 76.93it/s]


In [1053]:
pdp_experience = exp.model_profile(variables=['company_type'], variable_type='categorical')
pdp_experience.plot(title="PDP for company_type")

Calculating ceteris paribus: 100%|██████████| 1/1 [00:00<00:00, 76.93it/s]


In [1054]:
pdp_experience = exp.model_profile(variables=['last_new_job'], variable_type='categorical')
pdp_experience.plot(title="PDP for last_new_job")

Calculating ceteris paribus: 100%|██████████| 1/1 [00:00<00:00, 76.93it/s]


Wartości SHAP

In [1055]:
obs = X_train.iloc[11]

In [1056]:
prediction = model.predict_proba(obs.to_frame().T)
print(prediction)

[[0.7519032 0.2480968]]


In [1057]:
bd1 = exp.predict_parts(obs, type='break_down_interactions', order=['relevent_experience', 'gender', 'enrolled_university'])
bd2 = exp.predict_parts(obs, type='break_down_interactions', order=['gender', 'enrolled_university', 'relevent_experience'])
bd3 = exp.predict_parts(obs, type='break_down_interactions', order=['enrolled_university', 'relevent_experience', 'gender'])

In [1064]:
plt.figure(figsize=(15, 5))
bd1.plot()
bd2.plot()
bd3.plot()
plt.tight_layout()
plt.show()

<Figure size 1500x500 with 0 Axes>

In [1059]:
shap = exp.predict_parts(obs, type='shap')

In [1063]:
plt.figure(figsize=(10, 4))
shap.plot()
shap.plot(max_vars=10)
plt.tight_layout()
plt.show()

<Figure size 1000x400 with 0 Axes>